<a href="https://colab.research.google.com/github/ntolayd/CE888/blob/main/Assignment/forestfire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip "/content/gdrive/My Drive/Test.zip"
!unzip "/content/gdrive/My Drive/Training.zip"

In [3]:
import numpy as np
from tqdm import tqdm
from random import shuffle
import cv2
import os 
from keras.preprocessing.image import ImageDataGenerator 
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
Train_fire_dir = 'Training/Fire'
Train_nofire_dir = 'Training/No_Fire'
Test_fire_dir = 'Test/Fire'
Test_nofire_dir = 'Test/No_Fire'
Train_path = 'Training'
Test_path = 'Test'

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    Train_path,
    target_size=(254, 254),
    batch_size=32,
    class_mode='binary',
    subset='training',shuffle=True) # set as training data

validation_generator = train_datagen.flow_from_directory(
    Train_path, # same directory as training data
    target_size=(254, 254),
    batch_size=32,
    class_mode='binary',
    subset='validation',shuffle=True) # set as validation data

test_generator = test_datagen.flow_from_directory(
        Test_path,
        batch_size=32,
        class_mode='binary', shuffle=False)

Found 31501 images belonging to 2 classes.
Found 7874 images belonging to 2 classes.
Found 8617 images belonging to 2 classes.


In [7]:
from keras.applications.densenet import DenseNet121
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dropout
from keras.layers import Dense

densenet = DenseNet121(
    include_top=False,
    input_shape=(254,254,3))
model = Sequential()
model.add(densenet)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


29089792/29084464 [==============================] - 0s 0us/step


In [8]:
from keras.optimizers import Adam
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.00005),
              metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 8, 8, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 7,038,529
Trainable params: 6,954,881
Non-trainable params: 83,648
_________________________________________________________________


In [10]:
history = model.fit_generator(
          train_generator,
          steps_per_epoch = train_generator.samples // 32,
          validation_data = validation_generator, 
          validation_steps = validation_generator.samples // 32,
          epochs = 5)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
984/984 [==============================] - 477s 467ms/step - loss: 0.0693 - accuracy: 0.9716 - val_loss: 0.0241 - val_accuracy: 0.9949
Epoch 2/5
984/984 [==============================] - 471s 478ms/step - loss: 0.0019 - accuracy: 0.9996 - val_loss: 0.0672 - val_accuracy: 0.9804
Epoch 3/5
984/984 [==============================] - 470s 478ms/step - loss: 0.0019 - accuracy: 0.9995 - val_loss: 0.0309 - val_accuracy: 0.9928
Epoch 4/5
984/984 [==============================] - 471s 479ms/step - loss: 0.0015 - accuracy: 0.9997 - val_loss: 0.0530 - val_accuracy: 0.9841
Epoch 5/5
984/984 [==============================] - 472s 479ms/step - loss: 1.8754e-04 - accuracy: 1.0000 - val_loss: 0.0746 - val_accuracy: 0.9820


In [11]:
predictions = model.predict_generator(test_generator)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [14]:
# import classification_report
from sklearn.metrics import classification_report

# get the ground truth of your data. 
test_labels=test_generator.classes 

# predict the probability distribution of the data

# get the class with highest probability for each sample
y_pred = [1 * (x[0]>=0.5) for x in predictions]

# get the classification report
print(classification_report(test_labels, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.88      0.90      5137
           1       0.84      0.89      0.86      3480

    accuracy                           0.88      8617
   macro avg       0.88      0.88      0.88      8617
weighted avg       0.89      0.88      0.88      8617

